In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Subset
from transformers import AutoModel
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
from collections import Counter
import torch.nn.functional as F
import torchvision.models as models
from torchmetrics.classification import AUROC
import random
from torch.utils.data import Dataset
from PIL import Image
random.seed(123)

/opt/anaconda3/envs/pneumonia2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 1e-4
VAL_SPLIT = 0.2  # 80% train, 20% validation

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [4]:
dataset = datasets.ImageFolder(root="pneumonia/train", transform=transform)

labels = np.array([sample[1] for sample in dataset.samples])  # Extract labels from dataset

# Train-validation split with stratification
train_indices, val_indices = train_test_split(
    np.arange(len(labels)), test_size=VAL_SPLIT, stratify=labels, random_state=42
)

# Create subset datasets
train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Print class mapping
print(f"Class Mapping: {dataset.class_to_idx}")

Class Mapping: {'NORMAL': 0, 'PNEUMONIA': 1}


In [5]:
class PneumoniaModel(nn.Module):
    def __init__(self, num_classes=2):
        super(PneumoniaModel, self).__init__()
        self.backbone = models.convnext_base(weights="IMAGENET1K_V1")  # Use "convnext_small" if needed


        for param in self.backbone.features.parameters():
            param.requires_grad = False

        in_features = self.backbone.classifier[2].in_features
        self.backbone.classifier[2] = nn.Linear(in_features, 1)

    def forward(self, x):
        logits = self.backbone(x)  
        probs = torch.sigmoid(logits) 
        return probs 


In [6]:
model = PneumoniaModel(num_classes=2).to(DEVICE)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.backbone.classifier.parameters(), lr=LEARNING_RATE)

In [7]:
def evaluate(model, val_loader):
    model.eval() 
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1) 
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    return accuracy


In [8]:
def evaluate(model, val_loader):
    model.eval() 
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1) 
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    return accuracy


In [9]:
def train(model, train_loader, val_loader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE).float() 

            optimizer.zero_grad()
            outputs = model(images).squeeze(1) 
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            preds = (torch.sigmoid(outputs) > 0.5).float()  
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = 100 * correct / total
        val_auroc = evaluate_auroc(model, val_loader)

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%, Val AUROC: {val_auroc:.4f}")

In [10]:
auroc_metric = AUROC(task="binary")  # Use "multiclass" for >2 classes

def evaluate_auroc(model, val_loader):
    model.eval()
    probs_list = []
    labels_list = []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            outputs = model(images).squeeze(1)
            probs = torch.sigmoid(outputs) 

            probs_list.append(probs.cpu())
            labels_list.append(labels.cpu())

    probs_list = torch.cat(probs_list)
    labels_list = torch.cat(labels_list)

    auroc_score = auroc_metric(probs_list, labels_list)
    return auroc_score.item()

In [11]:
train(model, train_loader, val_loader, criterion, optimizer, epochs=10)


Epoch 1/10, Loss: 0.5560, Train Acc: 74.28%, Val AUROC: 0.9684
Epoch 2/10, Loss: 0.4953, Train Acc: 74.28%, Val AUROC: 0.9725
Epoch 3/10, Loss: 0.4734, Train Acc: 74.28%, Val AUROC: 0.9761
Epoch 4/10, Loss: 0.4629, Train Acc: 74.28%, Val AUROC: 0.9786
Epoch 5/10, Loss: 0.4565, Train Acc: 74.28%, Val AUROC: 0.9807
Epoch 6/10, Loss: 0.4522, Train Acc: 74.28%, Val AUROC: 0.9821
Epoch 7/10, Loss: 0.4489, Train Acc: 74.28%, Val AUROC: 0.9834
Epoch 8/10, Loss: 0.4463, Train Acc: 74.28%, Val AUROC: 0.9844
Epoch 9/10, Loss: 0.4443, Train Acc: 74.28%, Val AUROC: 0.9853
Epoch 10/10, Loss: 0.4425, Train Acc: 74.28%, Val AUROC: 0.9862


In [13]:
class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.image_paths = [os.path.join(root_dir, fname) for fname in os.listdir(root_dir) if fname.endswith(('.jpg', '.png', '.jpeg'))]
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB") 

        if self.transform:
            image = self.transform(image)

        return image, img_path 

test_dir = "pneumonia/test"
test_dataset = TestDataset(test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

print(f"Loaded {len(test_dataset)} test images.")

Loaded 624 test images.


In [15]:
def predict_test_set(model, test_loader):
    model.eval()
    predictions = []

    with torch.no_grad():
        for images, image_paths in test_loader: 
            images = images.to(DEVICE)

            outputs = model(images) 
            probs = torch.sigmoid(outputs).squeeze(1) 

            filenames = [os.path.basename(path) for path in image_paths]
            predictions.extend(list(zip(filenames, probs.cpu().numpy())))
    predictions.sort(key=lambda x: int(os.path.splitext(x[0])[0])) 
    return predictions

test_preds = predict_test_set(model, test_loader)

df = pd.DataFrame(test_preds, columns=["Id", "Category"])
df.to_csv("convNetXt_IMAGENET1K_V1__test_predictions.csv", index=False)

print("Test predictions saved")

Test predictions saved


In [16]:
torch.save(model.state_dict(), "convNetXt_IMAGENET1K_V1.pth")